# 03 - モデル登録と学習、MLflow 実験記録

このノートブックでは、未学習の PointNet モデルを MLflow レジストリに登録し、
その後登録済みモデルをロードして SemanticKITTI データでファインチューニングを行います。

In [ ]:
import os
import sys
import importlib.util
import importlib

# setuptools の _distutils が使われるように強制する
os.environ["SETUPTOOLS_USE_DISTUTILS"] = "local"

# setuptools を先にインポート
import setuptools

# 既存のdistutilsをクリア
if "distutils" in sys.modules:
    del sys.modules["distutils"]
    
# _distutils_hackのdo_override関数をバックアップし、正しく動作するように修正
if "_distutils_hack" in sys.modules:
    original_do_override = sys.modules["_distutils_hack"].do_override
    
    def patched_do_override():
        try:
            return original_do_override()
        except AssertionError:
            # distutilsのパスを調整
            import distutils
            distutils.__path__.insert(0, setuptools.__path__[0] + "/_distutils")
            return True
            
    sys.modules["_distutils_hack"].do_override = patched_do_override

In [ ]:
import sys
import os
# プロジェクトルートをパスへ追加
sys.path.append(os.path.abspath('..'))

## 1. ベースモデルの登録

In [ ]:
import mlflow
import mlflow.pytorch
from ml.models.pointnet import PointNetSeg

# 実験設定
mlflow.set_experiment('PointNetBaseRegistration')
with mlflow.start_run(run_name='register-base') as run:
    # 未学習モデルインスタンス化
    base_model = PointNetSeg(num_classes=90)
    # MLflowにモデルをログ
    mlflow.pytorch.log_model(base_model, 'base_model')
    # レジストリ登録
    model_uri = f'runs:/{run.info.run_id}/base_model'
    registered = mlflow.register_model(model_uri=model_uri, name='PointNetBase')
    print(f'Registered model: {registered.name} v{registered.version}')

## 2. モデルのロード & ファインチューニング

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import glob
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.pytorch
from ml.train.train_pointnet import SemanticKittiDataset

# 点群データでは、サンプルごとに点の数が異なるのが一般的。
# 一方、PyTorchのデフォルトのcollate関数は、すべてのテンソルが同じサイズであることを期待する。
# よって、今回はカスタムcollate関数を定義
def pointcloud_collate(batch):
    """点群データ用のカスタムcollate関数。異なるサイズの点群を処理します。"""
    # バッチから点群とラベルを抽出
    points = [item[0] for item in batch]  # points: list of [3, N_i]
    labels = [item[1] for item in batch]  # labels: list of [N_i]
    
    # すべての点群から均一にサンプリングする点の数を決定
    # 最小の点数を使用するか、固定値を使用
    min_points = min([p.shape[1] for p in points])
    target_points = min(min_points, 10000)  # 10000点を最大とする
    
    # 各点群から均一にサンプリング
    sampled_points = []
    sampled_labels = []
    
    for pts, lbls in zip(points, labels):
        if pts.shape[1] > target_points:
            # インデックスをランダムにサンプリング
            idx = torch.randperm(pts.shape[1])[:target_points]
            sampled_points.append(pts[:, idx])
            sampled_labels.append(lbls[idx])
        else:
            # 点が少ない場合はそのまま使用
            sampled_points.append(pts)
            sampled_labels.append(lbls)
    
    # テンソルをスタック
    points_batch = torch.stack(sampled_points)
    labels_batch = torch.stack(sampled_labels)
    
    return points_batch, labels_batch

# 実験設定
mlflow.set_experiment('PointNetFineTune')
with mlflow.start_run(run_name='finetune-run') as run:
    # ベースモデルのロード
    model = mlflow.pytorch.load_model('models:/PointNetBase/1')
    model.train()

    # データローダ作成
    dataset = SemanticKittiDataset('data/preprocessed')
    loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=pointcloud_collate)

    # オプティマイザ・損失関数
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    criterion = nn.CrossEntropyLoss()

    all_preds, all_targets = [], []
    epochs = 5
    for epoch in range(epochs):
        total_loss = 0.0
        for points, labels in loader:
            # [B,3,N] と [B,N]
            targets = labels[:,0]
            outputs = model(points)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            all_preds.extend(outputs.argmax(dim=1).tolist())
            all_targets.extend(targets.tolist())
        avg_loss = total_loss / len(loader)
        mlflow.log_metric('epoch_loss', avg_loss, step=epoch)
        print(f'Epoch {epoch+1}/{epochs}, loss={avg_loss:.4f}')

    acc = accuracy_score(all_targets, all_preds)
    mlflow.log_metric('final_accuracy', acc)
    print(f'Final Accuracy: {acc:.4f}')

    # ファインチューニング済みモデルをログ & 登録
    mlflow.pytorch.log_model(model, 'finetuned_pointnet')
    mlflow.register_model(f'runs:/{run.info.run_id}/finetuned_pointnet', 'PointNetFineTuned')
    print('Fine-tuned model registered as PointNetFineTuned')